In [1]:
import os
import sys
import time
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy

import torch
from torch_geometric.loader import DataLoader, ClusterData, ClusterLoader, NeighborLoader, DataListLoader
from torch_geometric.data import Batch
from torch_geometric.nn import DataParallel
import graphClasses as gf
import edgePruning as ep
import visFunctions as vf

import json
import pickle
import importlib

/opt/conda/envs/geo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"Let's use {torch.cuda.device_count()} GPUs!")

Let's use 4 GPUs!


In [3]:
my_seed = 830

np.random.seed(my_seed)
torch.manual_seed(my_seed)
torch.cuda.manual_seed(my_seed)
torch.backends.cudnn.deterministic = True

# Active Learning

Learn models as data is accessed from the pool via the specified query selection method.

The 100 meshes in the MPI dataset is actually comprised of 10 samples each wih 10 poses. To reduce computational cost but to preserve logic given the nature of this dataset, our active learning queries all poses from the patient selected at each round. Specifically, we sum the disagreement from all poses for each patient, and select the patient with highest total disagreement across poses. We query all poses from this patient for the next round of active learning.

In [4]:
def NewOnlineLearning(dim_h, dim_in, model_dir, pred_dir, gif_dir, start_n, end_n, num_instances, num_poses, querySelection, X, epochs, rand_i,
                      load_model:str=None,mc_repeats:int=10,mc_prob:float=0.3,):
    train_accs = np.zeros((num_instances+1))
    train_dices = np.zeros((num_instances+1))
    train_ious = np.zeros((num_instances+1))
    train_losses = np.zeros((num_instances+1))

    unobs_accs = np.zeros((num_instances+1))
    unobs_dices = np.zeros((num_instances+1))
    unobs_ious = np.zeros((num_instances+1))
    unobs_losses = np.zeros((num_instances+1))
    
    X_L, X_U = X[:start_n], X[start_n:]
    print('X_L,X_U',len(X_L), len(X_U))
    print(len(X_L[0]), len(X_U[0]))

    if run_i == 0:
        idx_list = list(range((10-start_n)*num_poses))
#         print(len(idx_list))
        imgobj = vf.ImageObject(gif_dir,idx_list)

    model_fname = f'/al_base'
    model_path = model_dir+model_fname

    model = gf.GraphSAGE(dim_h, dim_in, 12)
    model = DataParallel(model)
    model.to(device)

    print("="*30, f'Starting {str(querySelection).upper()}','='*30)

    print(f'Training/Testing on Initial sampling..........')
    
    # Train/test first samples
    #split data in test, train and validation
    val_train_split = len(X_L)//3
    val_lists = X_L[:val_train_split]
    val_list = [pose for patient_poses in val_lists for pose in patient_poses]
    # print('Constructing validation data.')	
    # val_batch = Batch.from_data_list(val_list)
    # val_data = DataLoader(val_batch, batch_size=1)
    val_data = DataListLoader(val_list, batch_size=6, shuffle=True)

    train_lists = X_L[val_train_split:]
    train_list = [pose for patient_poses in train_lists for pose in patient_poses]
    # print('Constructing training data.')
    # train_batch = Batch.from_data_list(train_list)
    # train_data = DataLoader(train_batch, batch_size=1)
    train_data = DataListLoader(train_list, batch_size=6, shuffle=True)
    
    test_list = [pose for patient_poses in X_U for pose in patient_poses]
    # print('Constructing test data.')
    # test_data = DataLoader(test_list)
    test_data = DataListLoader(test_list, batch_size=6, shuffle=True)
    # train_data = DataListLoader(train_list, batch_size=1, shuffle=True)
    # val_data = DataListLoader(val_list, batch_size=1, shuffle=True)
    
    print('Training model...')
    start_time = time.time()
    train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    train_accs[0] = train_acc
    train_dices[0] = train_dice
    train_losses[0] = train_loss
    train_ious[0] = train_iou
    print(f'Total time taken for training: {time.time()-start_time:.4f} seconds')
    
    print('Saving model...')
    gf.save_model(model, model_path)
    
    print('Running test graphs.')
    loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
    unobs_accs[0] = acc
    unobs_dices[0] = dice
    unobs_losses[0] = loss
    unobs_ious[0] = iou
    # print('passive predict_probs type',type(predict_probs))
    print(predict_probs.shape)
    print(f'Testing loss: {loss:.2f}')
    print(f'Testing accuracy: {acc:.2f}')
    print(f'Testing iou: {iou:.2f}')
    print(f'Testing dice: {dice:.2f}')
    print('-'*40)

    
    f = open(os.path.join(pred_dir,f'al_base.pckl'),'wb')
    pickle.dump(predict_probs, f)
    f.close()


    if run_i == 0:
        if querySelection is not None:
            imgobj.create_images(test_list, predict_probs, num_instances, num_poses, 'initial', querySelection.upper())
            imgobj.create_images(test_list, predict_probs, num_instances, num_poses, 'initial', 'Passive')

    for instance_i, split_i in enumerate(range(start_n,end_n)):
        if querySelection is not None:
            print(f'\n>>> Querying instance {instance_i+1} with {str(querySelection).upper()}..........')
        else:
            print(f'\n>>> Querying instance {instance_i+1} with Passive..........')

        if querySelection == None:

            model_fname = f'/al_instance{instance_i}'
            model_path = model_dir+model_fname

            # Passive learning
            X_L = X[:split_i+1]
            X_U = X[split_i+1:]
            xstar_i = 0

            print(f'PASSIVE QUERIED INDEX: {xstar_i}')        
            
            # unobs_accs[instance_i] = acc

        elif querySelection == "qbc":
            
            ####################################
            committee_size = 3
            
            labeled_list = [pose for patient_poses in X_L for pose in patient_poses]
            
            X_Ls = Bootstrap(committee_size, labeled_list)
            print(len(X_Ls), len(X_Ls[0]), type(X_Ls[0][0]))

            print("Training query committee.....")
            xstar_i, _, _, consensus_probs = QBC(dim_h, dim_in, device, X_Ls, X_U, instance_i, epochs)

            print(f'QBC QUERIED INDEX: {xstar_i}')
            # print('QBC consensus_probs type',type(consensus_probs))

            
            # test_list = [pose for patient_poses in X_U for pose in patient_poses]
            # if run_i == 0:
            #     imgobj.create_images(test_list, torch.tensor(consensus_probs), num_instances, num_poses, instance_i, querySelection)
            #     imgobj.track_index(xstar_i,num_poses)
                
            #####################################
            
            X_L = X_L + [X_U[xstar_i]]
            X_U = X_U[:xstar_i] + X_U[xstar_i+1:]
            print(len(X_L), len(X_U))
# #             print(len(X_L[0]), len(X_U[0]))

        elif querySelection == "mc":
            # model = gf.GraphSAGE_mc(dim_h, dim_in, 12, mc_prob=mc_prob)

            xstar_i = MC_dropout(dim_h, dim_in, device, X_L, X_U, num_poses, epochs,mc_prob,mc_repeats)
            print(f'MCD QUERIED INDEX: {xstar_i}')

            X_L = X_L + [X_U[xstar_i]]
            X_U = X_U[:xstar_i] + X_U[xstar_i+1:]

        print('X_L,X_U',len(X_L), len(X_U))
        # print(len(X_L[0]), len(X_U[0]))

        model = gf.GraphSAGE(dim_h, dim_in, 12)
        model = DataParallel(model)
        model.to(device)

        #split data in test, train and validation
        val_train_split = len(X_L)//3
        val_lists = X_L[:val_train_split]
        val_list = [pose for patient_poses in val_lists for pose in patient_poses]
        # print('Constructing validation data.')	
        # val_batch = Batch.from_data_list(val_list)
        # val_data = DataLoader(val_batch, batch_size=1)
        val_data = DataListLoader(val_list, batch_size=6, shuffle=True)

        train_lists = X_L[val_train_split:]
        train_list = [pose for patient_poses in train_lists for pose in patient_poses]
        # print('Constructing training data.')
        # train_batch = Batch.from_data_list(train_list)
        # train_data = DataLoader(train_batch, batch_size=6)
        train_data = DataListLoader(train_list, batch_size=6, shuffle=True)

        test_list = [pose for patient_poses in X_U for pose in patient_poses]
        # print('Constructing test data.')
        # test_data = DataLoader(test_list)
        test_data = DataListLoader(test_list, batch_size=6, shuffle=True)
        
        print('Training model...')
        start_time = time.time()
        # train_loss, train_acc, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
        train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
        train_accs[instance_i] = train_acc
        print(f'Total time taken for training: {time.time()-start_time:0.4f} seconds')
        
        print('Saving model...')
        gf.save_model(model, model_path)
        
        print('Running test graphs.')
        # loss, acc, predict_probs = gf.test_model(model, device, test_data)
        loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
        # print('passive predict_probs type',type(predict_probs))
        print(f'Testing loss: {loss:.2f}')
        print(f'Testing accuracy: {acc:.2f}')
        print(f'Testing iou: {iou:.2f}')
        print(f'Testing dice: {dice:.2f}')
        print('-'*40)


        f = open(os.path.join(pred_dir,f'al_instance{instance_i}.pckl'),'wb')
        pickle.dump(predict_probs, f)
        f.close()
        print(predict_probs.shape)

        train_accs[instance_i+1] = train_acc
        train_dices[instance_i+1] = train_dice
        train_losses[instance_i+1] = train_loss
        train_ious[instance_i+1] = train_iou

        unobs_accs[instance_i+1] = acc
        unobs_dices[instance_i+1] = dice
        unobs_losses[instance_i+1] = loss
        unobs_ious[instance_i+1] = iou

        if run_i == 0:
            imgobj.track_index(xstar_i,num_poses)
            if querySelection is not None:
                imgobj.create_images(test_list, predict_probs, num_instances, num_poses, instance_i, querySelection.upper())
            else:
                imgobj.create_images(test_list, predict_probs, num_instances, num_poses, instance_i, 'Passive')
            # imgobj.track_index(xstar_i,num_poses)
            
            
    return np.array([train_accs, train_dices, train_losses, train_ious]) , np.array([unobs_accs, unobs_dices, unobs_losses, unobs_ious])


## Query By Committee

Committee size is 3.

**Bootstrapping.** Each bootstrapped dataset contains datasets of the same size as the training set at each round. Given a training set, $D$ of size $n$, Bagging involves creating $m$ new training sets of size $n$ by sampling from $D$ uniformly and with replacement (i.e., bootstrapping)

**Quanify Disagreement across committee for each sample.** Our model outputs are log softmax probabilities for class, for each node, for each testing mesh; thus the committee is a tensor with shape [$m$, size of test set, number of nodes, number of classes]. In our case, the shape in the first round of active learning is [5, 50, 6890, 12]. The consensus is the average probability of each class and has shape [50, 6890, 12]. 

In [5]:
def Bootstrap(cmt_size, X_L):
    # print(len(X_L))
    X_Ls = []
    numsamples = len(X_L)
    for _ in range(cmt_size):
        rand_i = np.random.choice(range(len(X_L)),size=numsamples,replace=True)
        
        X_Lbooted = [X_L[get_i] for get_i in rand_i]

        X_Ls.append(X_Lbooted)
    
    return X_Ls

In [6]:
def QuantifyLargestPatientDisagreement(probs_a, consensus, num_poses):
    '''
    Our active learning queries all poses from the patient selected. Pilot uses 3 poses per patient.
    '''
    
    # probs_a dimensions: number of committee members, number of testing meshes, number of nodes, number of classes
    # consensus dimensions: number of testing meshes, number of nodes, number of classes
    
#     disagreements = np.zeros((probs_a.shape[:-1]))
    
    print('Calculating committee disagreement...')
    print(probs_a.shape)
    print(consensus.shape)
    # print(np.sum(probs_a - np.expand_dims(consensus, axis=0)))
    x = np.nan_to_num(probs_a / consensus)
    disagreements = np.log(x) * probs_a
    # disagreements = np.log(probs_a / consensus) * probs_a
    print(disagreements.shape)
    print(type(disagreements))
    dis = np.sum(disagreements, axis=-1)
    print(dis.shape)
    dis2 = np.sum(dis, axis=0)
    print(dis2.shape)
    dis3 = np.sum(dis2, axis=-1)
    print(dis3.shape)
    print(dis3)
    istar = np.argmax(dis3)
    print(istar)
    i_star = istar // num_poses
    
#     Calculating committee disagreement...
#     (3, 50, 6890, 12)
#     (50, 6890, 12)
#     1.6078111064743439e-12
#     (3, 50, 6890, 12)
#     (3, 50, 6890)
#     (50, 6890)
#     (50,)
#     [12919.23067897 13268.70436478 12768.37933145 13087.97039782
#      12949.0351472  13002.58475957 12856.24169413 13334.29904475
#      12834.23435822 13230.1138174  12891.58814957 13209.63356772
#      12898.5849094  13238.97690748 12933.85406018 12871.79113584
#      12816.01247897 13253.71351485 12727.60596102 13384.16116318
#      12862.5137056  13159.11968156 12829.45098706 13170.80203524
#      12887.40786216 12865.17355183 12775.07284143 13219.37530298
#      12629.2226568  13337.91849373 12739.52346283 13126.33611794
#      12973.85461498 13142.88021733 12769.79374444 12892.50038965
#      12756.68478372 13331.63329016 12815.43654144 13258.23586064
#      12873.13341915 13089.28156523 12816.48535762 13246.32550649
#      12904.9296742  12894.91374903 12792.88232696 13576.56582084
#      12693.39705421 13276.45955528]
#     47
    

    return int(i_star)

In [7]:
def QuantifyLargestDisagreement(probs_a, consensus, num_poses):
    '''
    Our active learning queries all poses from the patient selected. Pilot uses 3 poses per patient.
    '''
    
    # probs_a dimensions: number of committee members, number of testing meshes, number of nodes, number of classes
    # consensus dimensions: number of testing meshes, number of nodes, number of classes
    
#     disagreements = np.zeros((probs_a.shape[:-1]))
    
    print('Calculating committee disagreement...')
    # print(probs_a.shape)
    # print(consensus.shape)
    # print(np.sum(probs_a - np.expand_dims(consensus, axis=0)))
    x = np.nan_to_num(probs_a / consensus)
    disagreements = np.log(x) * probs_a
    # disagreements = np.log(probs_a / consensus) * probs_a
    # print(disagreements.shape)
    # print(type(disagreements))
    dis = np.sum(disagreements, axis=-1)
    # print(dis.shape)
    dis2 = np.sum(dis, axis=0)
    # print(dis2.shape)
    dis3 = np.sum(dis2, axis=-1)
    # print(dis3.shape)
    # print(dis3)
    istar = np.argmax(dis3)
    # print(istar)
    i_star = istar // num_poses
    
#     Calculating committee disagreement...
#     (3, 50, 6890, 12)
#     (50, 6890, 12)
#     1.6078111064743439e-12
#     (3, 50, 6890, 12)
#     (3, 50, 6890)
#     (50, 6890)
#     (50,)
#     [12919.23067897 13268.70436478 12768.37933145 13087.97039782
#      12949.0351472  13002.58475957 12856.24169413 13334.29904475
#      12834.23435822 13230.1138174  12891.58814957 13209.63356772
#      12898.5849094  13238.97690748 12933.85406018 12871.79113584
#      12816.01247897 13253.71351485 12727.60596102 13384.16116318
#      12862.5137056  13159.11968156 12829.45098706 13170.80203524
#      12887.40786216 12865.17355183 12775.07284143 13219.37530298
#      12629.2226568  13337.91849373 12739.52346283 13126.33611794
#      12973.85461498 13142.88021733 12769.79374444 12892.50038965
#      12756.68478372 13331.63329016 12815.43654144 13258.23586064
#      12873.13341915 13089.28156523 12816.48535762 13246.32550649
#      12904.9296742  12894.91374903 12792.88232696 13576.56582084
#      12693.39705421 13276.45955528]
#     47
    

    return int(i_star)

In [8]:
def QBC(dim_h, dim_in, device, X_Ls, X_U, instance_i, epochs,load_model:str=None):
    
    # populate data matrix with dimensions: number of committee members, number of test patients * number of poses per patient, number of nodes, number of classes
    num_cmt = len(X_Ls)
    num_test_patient = len(X_U)
    num_patient_poses = len(X_U[0])
    probs_cmt = np.zeros((num_cmt, num_test_patient * num_patient_poses, X_U[0][0].num_nodes, 12))
    # print('probs_cmt.shape',probs_cmt.shape)
    
    val_accs = np.zeros((num_cmt))
    
    start_time_2 = time.time()
    
    # For each committee, 1) train, 2) test, 3) use log softmax for every pose to quantify disagreement among committee
    for i, x_l in enumerate(X_Ls):
        print(f'\n>>>>> Entering committe member {i+1}...')
        
        model_fname = f'/al_instance{instance_i}_cmt{i}'
        model_path = model_dir+model_fname
        
        if load_model == None:
            model = gf.GraphSAGE(dim_h, dim_in, 12)
            model = DataParallel(model)
            model.to(device)
        
        ################################
        #split data in train and validation
        #convert lists of lists
        
            print(len(x_l)) # 50 for 5 with 10 poses; 15 for 5 training patients with 3 poses each
            print(type(x_l[0])) # graph torch_geometric Data
        
            val_train_split = len(x_l)//3
            # print(val_train_split) # 16
            val_lists = x_l[:val_train_split]
            # print(len(val_lists)) # 16
            # print(len(val_lists[0])) # 4
            val_list = [patient_poses for patient_poses in val_lists]
            # print(len(val_list)) # 16
            # print(type(val_list[0])) # graph torch_geometric Data
            train_lists = x_l[val_train_split:]
            # print(len(train_lists)) # 34
            # print(len(train_lists[0])) # 4
            train_list = [patient_poses for patient_poses in train_lists]
            # print(len(train_list)) # 34
            # print(type(train_list[0])) # graph torch_geometric Data
            test_list = [pose for patient_poses in X_U for pose in patient_poses]
        
        else:
            train_list, val_list, test_list, model = load_old_save(model_path, load_model, num_poses)
            model.to(device)
            
        # print('Constructing training data.')
        # train_batch = Batch.from_data_list(train_list)
        # train_data = DataLoader(train_batch, batch_size=1)
        train_data = DataListLoader(train_list, batch_size=6, shuffle=True)

        # print('Constructing validation data.')
        # val_batch = Batch.from_data_list(val_list)
        # val_data = DataLoader(val_batch, batch_size=1)
        val_data = DataListLoader(val_list, batch_size=6, shuffle=True)

        print('Training model...')
        start_time = time.time()
        # train_loss, train_acc, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
        train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    
        # val_accs[i] = train_acc
#         train_accs[instance_i] = train_acc
        print(f'Total time taken for training: {time.time()-start_time:.4f}')

        # print('Saving model...')

        # gf.save_model(model, model_path)

        # print('Constructing test data.')
        # test_data = DataLoader(test_list)
        test_data = DataListLoader(test_list, batch_size=6, shuffle=True)

        # print('Running test graphs.')
        # loss, acc, predict_probs = gf.test_model(model, device, test_data)
        loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
#         print('Checking nans in predictions: where nans:', np.argwhere(np.isnan(predict_probs.cpu())))
#         print('predict_probs.shape', predict_probs.shape) # [50, 6890, 12] for 5 patients and 10 poses
#         print('predict_probs(0,:5,:)',predict_probs[0,:5,:]) # values are between 0 and 1
        
        probs_cmt[i,:,:,:] = predict_probs.cpu()

        # print(f'Testing loss: {loss:.2f}')
        # print(f'Testing accuracy: {acc:.2f}')
        # print('-'*40)
    
    print(f'Total time for training and testing committee memmber {i+1}: {time.time()-start_time_2:.4f} ')
    
    # Consensus is the average across committee
    # print('probs_cmt.shape = ', probs_cmt.shape) # (3, 50, 6890, 12) for 3 committe members
    consensus = np.mean(probs_cmt, axis=0)
    # print(f'percent zeros in consensus: {np.sum(consensus == 0) / consensus.size}')
#     print('consensus.shape',consensus.shape) # [50, 6890, 12] for test case with 5 patients and 10 poses
    
    xstar_i = QuantifyLargestDisagreement(probs_cmt, consensus, num_patient_poses)
    print(xstar_i)

    return xstar_i, np.mean(val_accs), acc, consensus

## Using MC dropout

**Number of repeats** Number of repeats for test run is 2

**Dropout probability** The probability of dropouts is 0.3

**Point selection** Entropy is used to select the next instance  

In [9]:
def MC_dropout(dim_h, dim_in, device, X_L, X_U, num_poses, epochs,mc_prob,mc_repeats,load_model:str=None):
    val_train_split = len(X_L)//3
    val_lists = X_L[:val_train_split]
    val_list = [pose for patient_poses in val_lists for pose in patient_poses]
    train_lists = X_L[val_train_split:]
    train_list = [pose for patient_poses in train_lists for pose in patient_poses]
    test_list = [pose for patient_poses in X_U for pose in patient_poses]
    
    model = gf.GraphSAGE_mc(dim_h, dim_in, 12, mc_prob=mc_prob)
    model = DataParallel(model)
    model.to(device)

    # print('Constructing training data.')
    # train_batch = Batch.from_data_list(train_list)
    # train_data = DataLoader(train_batch, batch_size=1)
    train_data = DataListLoader(train_list, batch_size=6, shuffle=True)

    # print('Constructing validation data.')	
    # val_batch = Batch.from_data_list(val_list)
    # val_data = DataLoader(val_batch, batch_size=1)
    val_data = DataListLoader(val_list, batch_size=6, shuffle=True)


    print('Training model...')
    start_time = time.time()
    # _, _, model = gf.train_model(model, device, train_data, val_data, epochs=epochs)
    train_loss, train_acc, train_iou, train_dice, model = gf.train_model(model, device, train_data, val_data,epochs=epochs)
    # train_accs[instance_i] = train_acc
    print(f'Total time taken for training: {time.time()-start_time:.4f} seconds')

    # print('Constructing test data.')
    # test_data = DataLoader(test_list)
    test_data = DataListLoader(test_list, batch_size=6, shuffle=True)

    #Repeat test runs
    print('Running test graphs.')
    # acc_list = []
    predict_probs_list = []
    start_time2 = time.time()
    for j in range(mc_repeats):
        # loss, acc, predict_probs = gf.test_model(model, device, test_data)
        loss, acc, iou, dice, predict_probs = gf.test_model(model, device, test_data)
        # acc_list.append(acc)
        predict_probs_list.append(predict_probs.cpu())
    
    print(f'Total time taken for perform MC predictions: {time.time()-start_time2:.4f} seconds')

    # print(f'Testing loss: {loss:.2f}')
    # print(f'Testing accuracy: {acc:.2f}')
    # print(f'Testing iou: {iou:.2f}')
    # print(f'Testing dice: {dice:.2f}')
    # print('-'*40)

    #produces mean value array of (# nodes, # classes)
    predict_probs_arr = np.mean(np.stack(predict_probs_list, axis=0), axis=0)
    # print('predict_probs_arr.shape ', predict_probs_arr.shape)
    
    #get best patient
    istar = Entropy(predict_probs_arr, num_poses)
    # istar = Entropy(predict_probs_arr, 10)

    return istar

In [10]:
def Entropy(probs_a, num_poses):
    '''
    Calculate best instance to select using entropy. 
    Paramters:
        probs_a: (u, c) numpy array
            u is the number of unlabeled points and c is the number of classes
    Returns:
        best_patient: int
            Index of best points in the unlabeled dataset
    
    '''
    
    # probs_a dimensions: number of unlabeled poses, number of nodes, number of classes
    entropy = np.sum(probs_a*np.log(probs_a), axis=-1)
    entropy = -np.sum(entropy, axis=-1)
    best_choice = np.argmax(entropy)
    best_patient = best_choice // num_poses
#     best_patient = best_choice   
    
    return best_patient

# Load data

Load MPI Faust meshes stored as .json dictionaries representing torch_geometric Data objects

In [15]:
num_poses = 10

gt_path = "segmentations.npz"
data = np.load(gt_path)
node_labels = data['segmentation_labels'].reshape(-1,)

save_dir = "/Data"
data_path = "./data_dicts"
data_light_path = "./data_dicts_light"

# graph_list = []
graph_files = os.listdir(data_path)
# print(len(graph_files))


# graph = gf.construct_graph(data_path, 0, node_labels)
# pruned_edge_index = ep.graph_pruning(graph)
# for i, g in enumerate(graph_files):
#     graph = gf.construct_graph(data_path, i, node_labels, edges=pruned_edge_index)
#     graph_list.append(graph)


nodes = np.random.choice(range(6890),size=int(6890*0.5), replace=False)
set_nodes = set(nodes)

with open(os.path.join(data_path, f'data_0.json'), 'r') as f:
    graph_dict = json.load(f)
edges = graph_dict['edge_index'] # list of 2 lists

light_edges = [[],[]]
for u,v in zip(edges[0], edges[1]):
    if (u in set_nodes) and (v in set_nodes):
#         print((u in set_nodes))
#         print('here')
        light_edges[0].append(u)
        light_edges[1].append(v)
        
temp_list = light_edges[0] + light_edges[1]
non_lone_nodes = list(np.unique(temp_list))
temp_list_mapped = [non_lone_nodes.index(n) for n in temp_list]
light_edges[0] = temp_list_mapped[:len(temp_list)//2]
light_edges[1] = temp_list_mapped[len(temp_list)//2:]

new_node_labels = node_labels[non_lone_nodes]
print(len(new_node_labels))

for i, graph in enumerate(graph_files):
    with open(os.path.join(data_path, f'data_{i}.json'), 'r') as f:
        graph_dict = json.load(f)
    graph_l = {}
    graph_l['x'] = [graph_dict['x'][x] for x in non_lone_nodes]
    graph_l['face'] = [] 
    graph_l['edge_index'] = light_edges
#     print(len(graph_dict['x']))
    # save json
    with open(os.path.join(data_light_path, f'data_{i}.json'), 'w') as f:
        json.dump(graph_l, f)

graph_list = []       
graph_files = os.listdir(data_light_path)
for i, graph in enumerate(graph_files):
    
    light_g = gf.construct_graph(data_light_path, i, new_node_labels)
    
    graph_list.append(light_g)



########## Patients ###################
# X = [graph_list[i*10:(i*10)+num_poses] for i in range(10)]

########## Poses ###################
X = []
for i in range(10):
    pose_list = []
    for j in range(10):
        pose_list.append(graph_list[i+(num_poses*j)])
    X.append(pose_list)


lens = [len(X[get_i]) for get_i in range(len(X))]
print(lens)
# X = [poses for poses in X[:8]]


3381
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [13]:
for i in range(10):
    x = ''
    for j in range(10):
        x += f'{i+(10*j)} '
        # print(i+(10*j))
    print(x)

0 10 20 30 40 50 60 70 80 90 
1 11 21 31 41 51 61 71 81 91 
2 12 22 32 42 52 62 72 82 92 
3 13 23 33 43 53 63 73 83 93 
4 14 24 34 44 54 64 74 84 94 
5 15 25 35 45 55 65 75 85 95 
6 16 26 36 46 56 66 76 86 96 
7 17 27 37 47 57 67 77 87 97 
8 18 28 38 48 58 68 78 88 98 
9 19 29 39 49 59 69 79 89 99 


# Main

Main simulation has 5 rounds.

In [16]:
num_runs = 5
# n = X.shape[0]
start_n = 3
end_n = 9
epochs = 25
dropout = 0.3
mc_repeats = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

num_instances = end_n - start_n

train_stats = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats = np.zeros((num_runs, 4, num_instances + 1))

train_stats_qbc = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats_qbc = np.zeros((num_runs, 4, num_instances + 1))

train_stats_mc = np.zeros((num_runs, 4, num_instances + 1))
unobs_stats_mc = np.zeros((num_runs, 4, num_instances + 1))

# train_accs = np.zeros((num_runs, num_instances + 1))
# unobs_accs = np.zeros((num_runs, num_instances + 1))

# train_accs_qbc = np.zeros((num_runs, num_instances + 1))
# unobs_accs_qbc = np.zeros((num_runs, num_instances + 1))

# train_accs_mc = np.zeros((num_runs, num_instances + 1))
# unobs_accs_mc = np.zeros((num_runs, num_instances + 1))

old_stdout = sys.stdout

sys.stdout = open(f"./logs/passive/passive_log_{start_n}_{end_n}.log", 'w')
# sys.stdout = open(f"./logs/qbc/qbc_log_{start_n}_{end_n}.log", 'w')
# sys.stdout = open(f"./logs/mc/mc_log_{start_n}_{end_n}.log", 'w')

begin_time = time.time()

for run_i in range(num_runs):
    sys.stdout.flush()

    np.random.seed(my_seed + run_i)
    torch.manual_seed(my_seed + run_i)
    torch.cuda.manual_seed(my_seed + run_i)
    torch.backends.cudnn.deterministic = True

    print()
    print('^'*40)
    print(f'Entering active learning Run {run_i+1}..........')

    rand_i = np.arange(len(X))
    np.random.shuffle(rand_i)
    X = [X[get_i] for get_i in rand_i]

    # dim_in = graph.num_features		
    # dim_h = graph.num_nodes
    dim_in = light_g.num_features		
    dim_h = light_g.num_nodes

    gif_dir = './save_gif2'
    if not os.path.exists(gif_dir):
        os.makedirs(gif_dir)

    # Passive Learning
    
    model_dir = f'./models/models_al_passive_debug2/al_run{run_i+1}'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    pred_dir = f'./preds/preds_al_passive_debug3/al_run{run_i+1}'
    if not os.path.exists(pred_dir):
        os.makedirs(pred_dir)
    train_pas, unobs_pas = NewOnlineLearning(dim_h, dim_in, model_dir, pred_dir, gif_dir, start_n, end_n, num_instances, num_poses, None, X,epochs,rand_i)
    # print(train_accs_j.shape)
    train_stats[run_i] = train_pas
    unobs_stats[run_i] = unobs_pas
    print("\n$$$$")
    print(f"Passive Run {run_i+1} train, unobs stats")
    print(train_pas)
    print(unobs_pas)
    print("$$$$\n")

    sys.stdout.flush()
    
    np.save(f'stats/passive_{start_n}_{end_n}_train_stats.npy',train_stats)
    np.save(f'stats/passive_{start_n}_{end_n}_unobs_stats.npy',unobs_stats)
    
    # Active Learning with query by committee
    
    model_dir = f'./models/models_al_qbc_debug3/al_run{run_i+1}'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    pred_dir = f'./preds/preds_al_qbc_debug3/al_run{run_i+1}'
    if not os.path.exists(pred_dir):
        os.makedirs(pred_dir)
    qbc_al_train, qbc_al_unobs = NewOnlineLearning(dim_h, dim_in, model_dir, pred_dir, gif_dir, start_n, end_n, num_instances, num_poses, "qbc", X, epochs, rand_i)
    train_stats_qbc[run_i] = qbc_al_train
    unobs_stats_qbc[run_i] = qbc_al_unobs

    print("\n$$$$")
    print(f"QBC Run {run_i+1} train, unobs stats")
    print(qbc_al_train)
    print(qbc_al_unobs)
    print("$$$$\n")

    np.save(f'stats/qbc_{start_n}_{end_n}_train_stats.npy',train_stats_qbc)
    np.save(f'stats/qbc_{start_n}_{end_n}_unobs_stats.npy',unobs_stats_qbc)
    
    sys.stdout.flush()

    
    # Active Learning with mc_dropout
    model_dir = f'./models/models_al_mc_debug3/al_run{run_i+1}'
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    pred_dir = f'./preds/preds_al_mc_debug3/al_run{run_i+1}'
    if not os.path.exists(pred_dir):
        os.makedirs(pred_dir)
    # train_accs, train_dices, train_losses, train_ious, unobs_accs, unobs_dices, unobs_losses, unobs_ious
    mc_al_train, mc_al_unobs = NewOnlineLearning(dim_h, dim_in, model_dir, pred_dir, gif_dir, start_n, end_n, 
                                                           num_instances, num_poses, "mc", X, epochs,rand_i,
                                                           mc_prob=dropout,mc_repeats=mc_repeats)
    # mc_al_train_accs, mc_al_unobs_accs, probs = OnlineLearning_mc(dim_h, dim_in, model_dir, pred_dir, gif_dir, start_n, end_n, num_instances, "mc", X, num_poses,
    #                                                       mc_prob=0.3,
    #                                                        mc_repeats=mc_repeats)
    train_stats_mc[run_i] = mc_al_train
    unobs_stats_mc[run_i] = mc_al_unobs

    print("\n$$$$")
    print(f"MC Run {run_i+1} train, unobs stats")
    print(mc_al_train)
    print(mc_al_unobs)
    print("$$$$\n")

    np.save(f'stats/mc_{start_n}_{end_n}_train_stats.npy',train_stats_mc)
    np.save(f'stats/mc_{start_n}_{end_n}_unobs_stats.npy',unobs_stats_mc)
    
    # sys.stdout.flush()
#     OnlineLearning_fake(dim_h, dim_in, model_dir, pred_dir, start_n, end_n, num_instances, None, X, run_i)
    
print(f'Total time: {(time.time()-begin_time)/60:.4f} mins')

sys.stdout.close()
sys.stdout = old_stdout



cuda


In [22]:
importlib.reload(vf)
importlib.reload(gf)

<module 'graphClasses' from '/home/ec2-user/finalproj/02750FinalProject/graphClasses.py'>

In [202]:
gif_dir = 'save_gif'
imgobj = vf.ImageObject(gif_dir,[])
imgobj.create_GIF(4, 7,'qbc', movie_name='movie3_qbc.gif')
imgobj.create_GIF(4, 7,'None', movie_name='movie3_passive.gif')
imgobj.create_GIF(4, 7,'mc', movie_name='movie3_mc.gif')

In [3]:
SAVE_DIR = 'outputs'
fname = 'al_debug_mc8.png'

if not os.path.isdir(SAVE_DIR):
    os.makedirs(SAVE_DIR)

fig = plt.figure(figsize = (10, 5))

x_labels=[]
for i in range(0,num_instances):
    x_labels.append(str(i+start_n))

datas = [train_accs, unobs_accs, train_accs_qbc, unobs_accs_qbc, train_accs_mc, unobs_accs_mc]
labels = ['Passive training', 'Passive unobserved', 'Active:QBC training', 'Active:QBC unobserved', 'Active:MC training', 'Active:MC unobserved']
colors = ['blue', 'xkcd:light blue', 'xkcd:green', 'xkcd:light green', 'xkcd:pink', 'xkcd:light pink']

# datas = [train_accs, unobs_accs, train_accs_mc, unobs_accs_mc]
# labels = ['Passive training', 'Passive unobserved', 'Active:MC training', 'Active:MC unobserved']
# colors = ['blue', 'xkcd:light blue', 'xkcd:pink', 'xkcd:light pink']

# # datas = [train_accs_qbc, unobs_accs_qbc]
# # labels = ['Active:MC training', 'Active:MC unobserved']
# # colors = ['xkcd:green', 'xkcd:light green']

for d, l, c in zip(datas, labels, colors):
    if len(d.shape) > 1:
        y_plot = np.mean(d, axis=0)
        sim_std = np.std(d, axis=0)
    else:
        y_plot = [np.mean(d)]*len(x_labels)
        sim_std = [np.std(d)]*len(x_labels)
    plt.plot(x_labels, y_plot, color =c, label=l)
    plt.errorbar(x_labels, y_plot, yerr=sim_std, color=c, ecolor=c, alpha=0.5, capsize=3)


plt.xlabel("Rounds")
# plt.xticks(fontsize=7,rotation=90)
plt.ylabel("accuracy")
plt.title("Learning curves")
plt.legend()
plt.show()
# plt.savefig(os.path.join(SAVE_DIR, fname))


NameError: name 'num_instances' is not defined

<Figure size 1000x500 with 0 Axes>

In [164]:
import importlib
importlib.reload(vf)

<module 'visFunctions' from '/home/sofialima/02750FinalProject/visFunctions.py'>

In [47]:
def OnlineLearning_fake(dim_h, dim_in, model_dir, pred_dir, start_n, end_n, num_instances, querySelection, X, run_i):
    train_accs = np.zeros((num_instances))
    unobs_accs = np.zeros((num_instances))
    
    X_L, X_U = X[:start_n], X[start_n:]
    
    if run_i == 0:
        save_path = r'./save_gif'
        idx_list = list(range(start_n, end_n))
        print(idx_list)
        imgobj = ImageObject(save_path,idx_list)

    for instance_i, split_i in enumerate(range(start_n,end_n)):
        print(f'Entering instance {instance_i+1}..........')
        
        if load_model == False:
            model = gf.GraphSAGE(dim_h, dim_in, 12)
            model.to(device)
            
        else:
            train_list, val_list, test_list, model = load_old_save(save_path, latest_model, num_poses)
            model.to(device)
            
        if querySelection == None:
            
            if load_model == False:

                # Passive learning
                X_i = X[:split_i]
            
                #split data in test, train and validation
                val_train_split = len(X_i)//3
                val_lists = X_i[:val_train_split]
                val_list = [pose for patient_poses in val_lists for pose in patient_poses]
                train_lists = X_i[val_train_split:]
                train_list = [pose for patient_poses in train_lists for pose in patient_poses]

                test_lists = X[split_i:]
                test_list = [pose for patient_poses in test_lists for pose in patient_poses]

            
            
            print('Constructing training data.')
            train_batch = Batch.from_data_list(train_list)
            train_data = DataLoader(train_batch, batch_size=1)

            print('Constructing validation data.')	
            val_batch = Batch.from_data_list(val_list)
            val_data = DataLoader(val_batch, batch_size=1)
            
            
            print('Constructing test data.')
            test_data = DataLoader(test_list)
            
            print('Running test graphs.')
            loss, acc, predict_probs = gf.test_model(model, device, test_data)
            print(predict_probs.shape)
            
            if run_i == 0:
                imgobj.create_images(test_list, predict_probs, instance_i, querySelection)
                print('split_i ',split_i)
                imgobj.track_index(split_i)

#                 imgobj.track_index(instance_i)
            

            print(f'Testing loss: {loss:.2f}')
            print(f'Testing accuracy: {acc:.2f}')
            print('-'*40)
            
            unobs_accs[instance_i] = acc